In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import sklearn.metrics as metric
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from xgboost import XGBClassifier
from sklearn.linear_model import Lasso, Ridge
import json
import joblib
import os

In [35]:
fpkm = pd.read_csv("../datasets/datasets_processed/fpkm/toden_fpkm.txt", sep="\t", index_col=0)
tpm = pd.read_csv("../datasets/datasets_processed/tpm/toden_tpm.txt", sep="\t", index_col=0)
raw_count = pd.read_csv("../datasets/datasets_raw/toden/toden_counts.txt", sep="\t")
meta = pd.read_excel("../datasets/datasets_raw/toden/toden_metadata.xlsx")

In [36]:
# Normalization: standard scale

def PreProcessNumerical(df, dfFit, columnNames):
    scaler = StandardScaler()
    scaler.fit(dfFit[columnNames])
    df[columnNames] = scaler.transform(df[columnNames])
    return df

In [37]:
# Normalize raw count data
preCount = raw_count.copy()
norCount = PreProcessNumerical(preCount, preCount, preCount.columns)

In [38]:
# Convert categorical variable ['Apoe.status','apoe_carrier','apoe_dose']
share_feat = ['Apoe.status','apoe_carrier','apoe_dose']
status_dic = {'None': 0, 'E2/E3': 1, 'E2/E4': 2, 'E3/E3': 3, 'E3/E4': 4, 'E4/E4': 5}
carrier_dic = {'None': 0, 'no_apoe4': 0, 'apoe4': 1}
dose_dic = {'apoe4': 1, 'None': 0, 'apoe44': 2, 'no_apoe4': 0}
com_dic = [status_dic, carrier_dic, dose_dic]

meta_feat = meta[share_feat].copy()
for i in range(len(share_feat)):
    meta_feat[share_feat[i]] = meta_feat[share_feat[i]].map(com_dic[i])
meta_feat.fillna(0, inplace=True)

In [99]:
tab1 = pd.read_excel("../datasets/abb1654_data_file_s1.xlsx", sheet_name="Upregulated genes", header=1)
tab2 = pd.read_excel("../datasets/abb1654_data_file_s1.xlsx", sheet_name="Downregulated genes", header=1)
tab3 = pd.read_csv('../datasets/hvg_genes.txt', header=None)
select_gene = list(set(list(tab1['ID'])+list(tab2['ID'])))
hvg_genes = tab3[0].tolist()

In [40]:
# Prepare training data (x)
df1 = norCount.T
df2 = df1.copy()
df3 = df1.copy()[list(set(df1.columns) & set(select_gene))]
df4 = df3.copy()
df5 = df1.copy()[list(set(df1.columns) & set(hvg_genes))]
df6 = df5.copy()

df7 = fpkm.copy()
df8 = fpkm.copy()
df9 = fpkm.copy()[list(set(df7.columns) & set(select_gene))]
df10 = df9.copy()
df11 = fpkm.copy()[list(set(df7.columns) & set(hvg_genes))]
df12 = df11.copy()

df13 = tpm.copy()
df14 = tpm.copy()
df15 = tpm.copy()[list(set(df13.columns) & set(select_gene))]
df16 = df15.copy()
df17 = tpm.copy()[list(set(df13.columns) & set(hvg_genes))]
df18 = df17.copy()

for i in meta_feat.columns:
    df2[i] = meta_feat[i].values
    df4[i] = meta_feat[i].values
    df6[i] = meta_feat[i].values
    df8[i] = meta_feat[i].values
    df10[i] = meta_feat[i].values
    df12[i] = meta_feat[i].values
    df14[i] = meta_feat[i].values
    df16[i] = meta_feat[i].values
    df18[i] = meta_feat[i].values

train_input=['count', 'count+apoe', 'select-gene count', 'select-gene count+apoe', 'hvg-gene count', 'hvg-gene count+apoe',
             'FPKM', 'FPKM+apoe', 'select-gene FPKM', 'select-gene FPKM+apoe', 'hvg-gene FPKM', 'hvg-gene FPKM+apoe',
             'TPM', 'TPM+apoe', 'select-gene TPM', 'select-gene TPM+apoe', 'hvg-gene TPM', 'hvg-gene TPM+apoe']
train_data = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18]

In [41]:
import warnings
warnings.filterwarnings('ignore')

# extract y
y = meta['Disease']
ma = {'NCI': 0, 'AD': 1}
for i in range(len(y)):
    y.iloc[i] = ma[y.iloc[i]]
y = y.astype(int)

In [42]:
def save_model(model, model_name):
    os.makedirs("../regression_models/", exist_ok=True)
    joblib.dump(model, os.path.join("../regression_models/", f"{model_name}.pkl"))

def load_model(model_name):
    return joblib.load(os.path.join("../regression_models/", f"{model_name}.pkl"))

In [9]:
def regression_models(train_data, train_input, y, model_type='linear', norm=None, results=None):
    for i in range(len(train_data)):
        dTrain = train_data[i]
        
        for k, (trainInd, valInd) in enumerate(KFold(shuffle=True, random_state=16).split(dTrain)):
            XTrain = dTrain.iloc[trainInd,]
            yTrain = y.iloc[trainInd,]
            XVal = dTrain.iloc[valInd,]
            yVal = y.iloc[valInd,]

            if model_type == 'linear' and norm == None:
                model = LinearRegression()
            elif model_type == 'logistic' and norm == None:
                model = LogisticRegression(max_iter=1000)
            elif model_type == 'linear' and norm == "l1":
                model = Lasso(alpha=1.0)
            elif model_type == 'linear' and norm == "l2":
                model = Ridge(alpha=1.0)
            elif model_type == 'logistic' and norm == "l1":
                model = LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear')
            elif model_type == 'logistic' and norm == "l2":
                model = LogisticRegression(max_iter=1000, penalty='l2')
                
            model.fit(XTrain, yTrain)
            pred = model.predict(XVal)
            pred_binary = (pred >= 0.5).astype(int)
            
            result_key = f"{model_type}_regression_{norm if norm else 'no'}_norm_{train_input[i]}_{k}"
            save_model(model, result_key)
            
            if model_type == 'linear':
                results[result_key] = {
                    "accuracy": metric.accuracy_score(yVal, pred_binary),
                    "roc_auc": metric.roc_auc_score(yVal, pred),
                    "f1_score": metric.f1_score(yVal, pred_binary),
                    "mcc_score": metric.matthews_corrcoef(yVal, pred_binary),
                    "recall": metric.recall_score(yVal, pred_binary)
                }
                feature_importance_df = pd.DataFrame({
                    'Feature': dTrain.columns,
                    'Importance': model.coef_
                }).sort_values(by='Importance', ascending=False)
                results[result_key]["feature_importance"] = feature_importance_df.to_dict(orient='records')
            else:
                results[result_key] = {
                    "accuracy": metric.accuracy_score(yVal, pred_binary),
                    "roc_auc": metric.roc_auc_score(yVal, model.predict_proba(XVal)[:,1]),
                    "f1_score": metric.f1_score(yVal, pred_binary),
                    "mcc_score": metric.matthews_corrcoef(yVal, pred_binary),
                    "recall": metric.recall_score(yVal, pred_binary)
                }
                feature_importance_df = pd.DataFrame({
                    'Feature': dTrain.columns,
                    'Importance': model.coef_[0]
                }).sort_values(by='Importance', ascending=False)
                results[result_key]["feature_importance"] = feature_importance_df.to_dict(orient='records')

            wrongly_predicted_indices = valInd[pred_binary != yVal.to_numpy()]
            results[result_key]["wrongly_predicted_indices"] = wrongly_predicted_indices.tolist()

In [10]:
results = {}
for i in ['linear', 'logistic']:
    regression_models(train_data, train_input, y, model_type=i, results=results)
    for j in ['l1', 'l2']:
        regression_models(train_data, train_input, y, model_type=i, norm=j, results=results)

In [47]:
def xgboost_model(train_data, train_input, y, norm=None, results=None):
    for i in range(len(train_data)):
        dTrain = train_data[i]
        
        for k, (trainInd, valInd) in enumerate(KFold(shuffle=True, random_state=16).split(dTrain)):
            XTrain = dTrain.iloc[trainInd]
            yTrain = y.iloc[trainInd]
            XVal = dTrain.iloc[valInd]
            yVal = y.iloc[valInd]

            if norm == None:
                model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
            elif norm == "l1":
                model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', reg_alpha=1)
            elif norm == "l2":
                model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', reg_lambda=1)
            
            model.fit(XTrain, yTrain)
            pred = model.predict(XVal)
            pred_binary = (pred >= 0.5).astype(int)

            result_key = f"xgboost_{norm if norm else 'no'}_norm_{train_input[i]}_{k}"
            save_model(model, result_key)

            results[result_key] = {
                "accuracy": metric.accuracy_score(yVal, pred_binary),
                "roc_auc": metric.roc_auc_score(yVal, model.predict_proba(XVal)[:,1]),
                "f1_score": metric.f1_score(yVal, pred_binary),
                "mcc_score": metric.matthews_corrcoef(yVal, pred_binary),
                "recall": metric.recall_score(yVal, pred_binary)
            }
            feature_importance_df = pd.DataFrame({
                'Feature': dTrain.columns,
                'Importance': model.feature_importances_
            }).sort_values(by='Importance', ascending=False)
            results[result_key]["feature_importance"] = feature_importance_df.to_dict(orient='records')

In [48]:
xgboost_model(train_data, train_input, y, results=results)
for i in ['l1', 'l2']:
    xgboost_model(train_data, train_input, y, norm=i, results=results)

In [94]:
def regression_model(train_data, train_input, y, model_type='linear', norm=None, results=None):
    
    dTrain = train_data[1]
    
    for k, (trainInd, valInd) in enumerate(KFold(shuffle=True, random_state=16).split(dTrain)):
        XTrain = dTrain.iloc[trainInd,]
        yTrain = y.iloc[trainInd,]
        XVal = dTrain.iloc[valInd,]
        yVal = y.iloc[valInd,]

        if model_type == 'linear' and norm == None:
            model = LinearRegression()
        elif model_type == 'logistic' and norm == None:
            model = LogisticRegression(max_iter=1000)
        elif model_type == 'linear' and norm == "l1":
            model = Lasso(alpha=1.0)
        elif model_type == 'linear' and norm == "l2":
            model = Ridge(alpha=1.0)
        elif model_type == 'logistic' and norm == "l1":
            model = LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear')
        elif model_type == 'logistic' and norm == "l2":
            model = LogisticRegression(max_iter=1000, penalty='l2')
            
        model.fit(XTrain, yTrain)
        pred = model.predict(XVal)
        pred_binary = (pred >= 0.5).astype(int)
        
        

        wrongly_predicted_indices = valInd[pred_binary != yVal.to_numpy()]
        return wrongly_predicted_indices.tolist()

In [95]:
results_new = {}
wrong = regression_model(train_data, train_input, y, model_type="logistic", results=results_new)
df2.index[wrong]

Index(['SRR10192321', 'SRR10192252', 'SRR10192230', 'SRR10192205',
       'SRR10192170', 'SRR10192171', 'SRR10192487', 'SRR10192478',
       'SRR10192353', 'SRR10192329'],
      dtype='object')

In [50]:
with open('regression_results.json', 'w') as f:
    json.dump(results, f, indent=4)

In [78]:
import csv
file_path = "regresssion_results.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=',')

    writer.writerow(["", "Model", "Penalty", "Normalization", "APOE", "Gene Subset", "Fold", "Accuracy", "AUC", "F1", "MCC", "Recall"])

    for key, value in results.items():
        split = key.split("_")
        row = [key[:-1]+str(int(split[-1])+1), split[0]+" "+split[1], split[2]+" "+split[3], split[4], int(split[5])+1] if len(split) == 6 else [key[:-1]+str(int(split[-1])+1), split[0], split[1]+" "+split[2], split[3], int(split[4])+1]
        new_row = [row[0], row[1], row[2][:2]]
        if('count' in row[3]):
            new_row.append('count')
        elif('FPKM' in row[3]):
            new_row.append('FPKM')
        else:
            new_row.append('TPM')
        if('apoe' in row[3]):
            new_row.append('True')
        else:
            new_row.append('False')
        if('select-gene' in row[3]):
            new_row.append('select-gene')
        elif('hvg-gene' in row[3]):
            new_row.append('hvg-gene')
        else:
            new_row.append('total')
        new_row.append(row[4])
        for k, v in value.items():
            if k != "feature_importance":
                new_row.append(v)
        writer.writerow(new_row)


In [98]:
import pandas as pd

file_path = "regression_results.csv"

df = pd.read_csv(file_path, delimiter=',')
df_sorted = df.sort_values(by='Accuracy', ascending=False)
df_sorted

,Unnamed: 0,Model,Penalty,Normalization,APOE,Gene Subset,Fold,Accuracy,AUC,F1,MCC,Recall
0,xgboost_no_norm_count_1,xgboost,no,count,False,total,1,0.925373,0.957143,0.927536,0.851026,0.914286
120,xgboost_l2_norm_count_1,xgboost,l2,count,False,total,1,0.925373,0.957143,0.927536,0.851026,0.914286
5,xgboost_no_norm_count+apoe_1,xgboost,no,count,True,total,1,0.925373,0.970536,0.927536,0.851026,0.914286
125,xgboost_l2_norm_count+apoe_1,xgboost,l2,count,True,total,1,0.925373,0.970536,0.927536,0.851026,0.914286
95,xgboost_l1_norm_count+apoe_1,xgboost,l1,count,True,total,1,0.910448,0.950893,0.911765,0.822480,0.885714
...,...,...,...,...,...,...,...,...,...,...,...,...
233,xgboost_l2_norm_hvg-gene FPKM_4,xgboost,l2,FPKM,False,hvg-gene,4,0.626866,0.731589,0.675325,0.260209,0.604651
263,xgboost_l2_norm_hvg-gene TPM_4,xgboost,l2,TPM,False,hvg-gene,4,0.626866,0.683140,0.683544,0.243203,0.627907
83,xgboost_no_norm_hvg-gene TPM_4,xgboost,no,TPM,False,hvg-gene,4,0.626866,0.683140,0.683544,0.243203,0.627907
183,xgboost_l1_norm_TPM_4,xgboost,l1,TPM,False,total,4,0.626866,0.697674,0.691358,0.226926,0.651163
